In [3]:
import os
import numpy as np
import pickle

# fed_batch

In [16]:
version = '220829_dFBA_exp_feed_v04'
loc     = f'/mnt/itching_scratch/mgotsmy/220829_slim_paper_v2/results/{version}.pkl'
q_pDNA_min = .1/100
special_interest_q_pDNA_max_levels = np.array([1,1.5,2,4])*q_pDNA_min
all_q_pDNA_max_levels = np.linspace(.1/100,.5/100,41)
# check if special_interest_q_pDNA_max_levels in all_q_pDNA_max_levels
for q in special_interest_q_pDNA_max_levels:
    if np.any(np.isclose(q,all_q_pDNA_max_levels)):
        pass
    else:
        print('q_pDNA_max = {:6.4f} mmol/(g h) from special interest not in all.'.format(q))

In [17]:
with open(loc,'rb') as file:
    combined_results = pickle.load(file)
found_q_pDNA_max_levels = list(combined_results.keys())
del file

In [18]:
# lookup stored files
q_pDNA_max_levels_set      = all_q_pDNA_max_levels
files_extra                = []
all_found_q_pDNA_max_levels = []
special_interest_found_q_pDNA_max_levels = []

# check if there are extra files or files missing
for q_pDNA_max in found_q_pDNA_max_levels:
    isclose = np.isclose(q_pDNA_max,q_pDNA_max_levels_set)
    if np.any(isclose):
        q_pDNA_max_levels_set = q_pDNA_max_levels_set[np.invert(isclose)]
        all_found_q_pDNA_max_levels.append(q_pDNA_max)
        if np.any(np.isclose(q_pDNA_max,special_interest_q_pDNA_max_levels)):
            special_interest_found_q_pDNA_max_levels.append(q_pDNA_max)
    else:
        files_extra.append([q_pDNA_max])

# print results
print('Reading files.')
if len(q_pDNA_max_levels_set):
    print('q_pDNA_max levels not found:')
    for q_pDNA_max in sorted(q_pDNA_max_levels_set):
        print('{:6.4f}'.format(q_pDNA_max))
else:
    print('All q_pDNA_max levels found.')
if len(files_extra):
    print('Extra q_pDNA_max levels found:')
    for q_pDNA_max in files_extra:
        print('{:6.4f}'.format(q_pDNA_max))
else:
    print('No extra q_pDNA_max levels found.')

Reading files.
All q_pDNA_max levels found.
No extra q_pDNA_max levels found.


In [19]:
def fed_batch_results_sorter(results):
    '''
    Sorts results dic according to numeric value in key.
    '''
    results_sorted = {}
    for S_0 in sorted(results):
        results_sorted[S_0] = results[S_0]
    return results_sorted

def fed_batch_unpack_values(sol):
    '''
    Unpacks concentrations in the sol.y array.
    '''
    t = sol.t
    V = sol.y.T[:,0]
    X = sol.y.T[:,1]
    S = sol.y.T[:,2]
    P = sol.y.T[:,3]
    G = sol.y.T[:,4]
    return t,V,X,S,P,G

def fed_batch_calculate_terminal_idx(sol):
    '''
    Calculates terminal_idx and start of starvation time.
    '''
    
    t,V,X,S,P,G = fed_batch_unpack_values(sol)
    
    # find t_starvation_start
    s_is_0 = np.isclose(S,0,atol=1e-3)
    if np.any(s_is_0):
        idx_star_start = np.where(s_is_0)[0][0]           # index of t_starvation_start
        t_star_start   = t[idx_star_start]                # value of t_starvation_start
        t_star_end     = t[-1]                            # value of t_starvation_end
        idx_star_end   = np.argmin(np.abs(t-t_star_end))  # index of t_starvation_end
        terminal_idx   = idx_star_end
    else:
        terminal_idx   = -1
        t_star_start   = np.nan
            
    return terminal_idx, t_star_start

def fed_batch_calculate_endpoints(results):
    '''
    Return concentrations of molecules of interest at the end of a process.
    '''
    
    endpoints = []
    for S_0 in sorted(results):
        sol = results[S_0]
        t,V,X,S,P,G = fed_batch_unpack_values(sol)
        terminal_idx, t_star_start = fed_batch_calculate_terminal_idx(sol)
        endpoint = [
            S_0,
            t[terminal_idx],
            V[terminal_idx],
            X[terminal_idx],
            S[terminal_idx],
            P[terminal_idx],
            G[terminal_idx],
            t_star_start
        ]
        endpoints.append(endpoint)
    return np.array(endpoints)

def fed_batch_trim_processes(results):
    '''
    Trim processes of special interest to the calculated process ends.
    '''
    
    trimmed_results = {}
    for S_0 in sorted(results):
        sol = results[S_0]
        t,V,X,S,P,G = fed_batch_unpack_values(sol)
        terminal_idx, t_star_start = fed_batch_calculate_terminal_idx(sol)
        curves = [
            S_0,
            t[:terminal_idx],
            V[:terminal_idx],
            X[:terminal_idx],
            S[:terminal_idx],
            P[:terminal_idx],
            G[:terminal_idx],
            t_star_start
        ]
        trimmed_results[S_0] = curves
    return trimmed_results

In [20]:
print('Processing files.')
processed = {}
special_interest = {}
for q_pDNA_max in all_found_q_pDNA_max_levels:
    results = combined_results[q_pDNA_max]
    processed[q_pDNA_max] = fed_batch_calculate_endpoints(results)
    if q_pDNA_max in special_interest_found_q_pDNA_max_levels:
        special_interest[q_pDNA_max] = fed_batch_trim_processes(results)
    # break
print('Done.')
print('Length processed        = {:3.0f}'.format(len(processed)))
print('Length special interest = {:3.0f}'.format(len(special_interest)))

Processing files.
Done.
Length processed        =  41
Length special interest =   4


In [21]:
write_loc = f'/home/users/mgotsmy/pdna/220000_notebooks/220829_slim_paper_v2/preprocessing/{version}.pkl'
with open(write_loc,'wb') as file:
    pickle.dump([processed,special_interest],file)
print('Saved processed file.')

Saved processed file.


## fed_batch v2

In [4]:
version = '220829_dFBA_linear_feed_v05'
loc     = f'/mnt/itching_scratch/mgotsmy/220829_slim_paper_v2/results/{version}.pkl'
q_pDNA_min = .1/100
special_interest_q_pDNA_max_levels = np.array([1,1.5,2,4])*q_pDNA_min
all_q_pDNA_max_levels = np.sort(np.append(np.linspace(.1,.11,10)/100,np.linspace(.1,.5,41)/100)) #np.linspace(.1/100,.5/100,41)
# check if special_interest_q_pDNA_max_levels in all_q_pDNA_max_levels
for q in special_interest_q_pDNA_max_levels:
    if np.any(np.isclose(q,all_q_pDNA_max_levels)):
        pass
    else:
        print('q_pDNA_max = {:6.4f} mmol/(g h) from special interest not in all.'.format(q))

In [5]:
with open(loc,'rb') as file:
    combined_results = pickle.load(file)
found_q_pDNA_max_levels = list(combined_results.keys())
del file

In [6]:
# lookup stored files
q_pDNA_max_levels_set      = all_q_pDNA_max_levels
files_extra                = []
all_found_q_pDNA_max_levels = []
special_interest_found_q_pDNA_max_levels = []

# check if there are extra files or files missing
for q_pDNA_max in found_q_pDNA_max_levels:
    isclose = np.isclose(q_pDNA_max,q_pDNA_max_levels_set)
    if np.any(isclose):
        q_pDNA_max_levels_set = q_pDNA_max_levels_set[np.invert(isclose)]
        all_found_q_pDNA_max_levels.append(q_pDNA_max)
        if np.any(np.isclose(q_pDNA_max,special_interest_q_pDNA_max_levels)):
            special_interest_found_q_pDNA_max_levels.append(q_pDNA_max)
    else:
        files_extra.append([q_pDNA_max])

# print results
print('Reading files.')
if len(q_pDNA_max_levels_set):
    print('q_pDNA_max levels not found:')
    for q_pDNA_max in sorted(q_pDNA_max_levels_set):
        print('{:6.4f}'.format(q_pDNA_max))
else:
    print('All q_pDNA_max levels found.')
if len(files_extra):
    print('Extra q_pDNA_max levels found:')
    for q_pDNA_max in files_extra:
        print('{:6.4f}'.format(q_pDNA_max))
else:
    print('No extra q_pDNA_max levels found.')

Reading files.
All q_pDNA_max levels found.
No extra q_pDNA_max levels found.


In [10]:
def fed_batch_results_sorter_v2(results):
    '''
    Sorts results dic according to numeric value in key.
    '''
    results_sorted = {}
    for S_0 in sorted(results):
        results_sorted[S_0] = results[S_0]
    return results_sorted

def fed_batch_unpack_values_v2(sol):
    '''
    Unpacks concentrations in the sol.y array.
    '''
    t = sol.t
    V = sol.y.T[:,0]
    X = sol.y.T[:,1]
    S = sol.y.T[:,2]
    P = sol.y.T[:,3]
    G = sol.y.T[:,4]
    return t,V,X,S,P,G

def fed_batch_calculate_terminal_idx_v2(sol):
    '''
    Calculates terminal_idx and start of starvation time.
    '''
    
    t,V,X,S,P,G = fed_batch_unpack_values_v2(sol)
    
    # find t_starvation_start
    s_is_0 = np.isclose(S,0,atol=1e-3)
    if np.any(s_is_0):
        idx_star_start = np.where(s_is_0)[0][0]           # index of t_starvation_start
        t_star_start   = t[idx_star_start]                # value of t_starvation_start
        t_star_end     = t[-1]                            # value of t_starvation_end
        idx_star_end   = np.argmin(np.abs(t-t_star_end))  # index of t_starvation_end
        terminal_idx   = idx_star_end
    else:
        terminal_idx   = -1
        t_star_start   = np.nan
            
    return terminal_idx, t_star_start

def fed_batch_calculate_endpoints_v2(results):
    '''
    Return concentrations of molecules of interest at the end of a process.
    '''
    
    tmp = results[list(results.keys())[-1]]
    t,V,X,S,P,G = fed_batch_unpack_values_v2(tmp)
    no_starv_productivity = P[-1]/V[-1]/t[-1]
    
    endpoints = []
    for S_0 in sorted(results):
        sol = results[S_0]
        terminal_idx, t_star_start = fed_batch_calculate_terminal_idx_v2(sol)
        
        t,V,X,S,P,G = fed_batch_unpack_values_v2(sol)
        # productivity_time_series = P/V/(t+1e-8)
        productivity_time_series = P/V/t[terminal_idx]
        t_end_min = t[np.argmin(np.abs(np.nan_to_num(productivity_time_series-no_starv_productivity,nan=1000)))]
        endpoint = [
            S_0,
            t[terminal_idx],
            V[terminal_idx],
            X[terminal_idx],
            S[terminal_idx],
            P[terminal_idx],
            G[terminal_idx],
            t_star_start,
            t_end_min
        ]
        endpoints.append(endpoint)
    return np.array(endpoints)

def fed_batch_trim_processes_v2(results):
    '''
    Trim processes of special interest to the calculated process ends.
    '''
    
    trimmed_results = {}
    for S_0 in sorted(results):
        sol = results[S_0]
        t,V,X,S,P,G = fed_batch_unpack_values_v2(sol)
        terminal_idx, t_star_start = fed_batch_calculate_terminal_idx_v2(sol)
        curves = [
            S_0,
            t[:terminal_idx],
            V[:terminal_idx],
            X[:terminal_idx],
            S[:terminal_idx],
            P[:terminal_idx],
            G[:terminal_idx],
            t_star_start
        ]
        trimmed_results[S_0] = curves
    return trimmed_results

In [11]:
print('Processing files.')
processed = {}
special_interest = {}
for q_pDNA_max in all_found_q_pDNA_max_levels:
    results = combined_results[q_pDNA_max]
    processed[q_pDNA_max] = fed_batch_calculate_endpoints_v2(results)
    if q_pDNA_max in special_interest_found_q_pDNA_max_levels:
        special_interest[q_pDNA_max] = fed_batch_trim_processes_v2(results)
    # break
print('Done.')
print('Length processed        = {:3.0f}'.format(len(processed)))
print('Length special interest = {:3.0f}'.format(len(special_interest)))

Processing files.
Done.
Length processed        =  49
Length special interest =   4


In [12]:
write_loc = f'/home/users/mgotsmy/pdna/220000_notebooks/220829_slim_paper_v2/preprocessing/{version}_v2.pkl'
with open(write_loc,'wb') as file:
    pickle.dump([processed,special_interest],file)
print('Saved processed file.')

Saved processed file.
